<a href="https://colab.research.google.com/github/jw-jin/2021-DAP/blob/main/RiotAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive # 구글 드라이브에서 파일 불러오기
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd

# 드라이브 경로 설정
ch_data_path = '/content/drive/MyDrive/ch_matchdata.csv' # 챌린저 매치데이터 파일 경로
gm_data_path = '/content/drive/MyDrive/gm_matchdata.csv' # 그랜드마스터 매치데이터 파일 경로
m_data_path = '/content/drive/MyDrive/m_matchdata.csv' # 마스터 매치데이터 파일 경로

# csv 불러오기
ch_data = pd.read_csv(ch_data_path, encoding= 'cp949')
gm_data = pd.read_csv(gm_data_path, encoding= 'cp949')
m_data = pd.read_csv(m_data_path, encoding= 'cp949')

In [141]:
pd.set_option('display.max_row', 100)
c_df = ch_data
# 챔피언별 승률 
champion_df = c_df.loc[:,['championId', 'championName']]

champion_df = champion_df.drop_duplicates() # 중복제거
champion_df = champion_df.sort_values('championId') # championid 순 정렬

id_list = champion_df['championId'].tolist()
id_dict = champion_df.set_index('championId').T.to_dict('list')
ch_win_df = pd.DataFrame(columns=['championId','championName', 'winrate', 'loserate'])

for key,value in id_dict.items(): # key = id value = name
  if(key < 888): #결측값 처리 (최대 키값은 887)
    tmp_df = c_df[c_df['championId'] == key] 
    win_count = tmp_df['win'].value_counts(normalize=True)
    data = {
        'championId' : key,
        'championName' : value[0],
        'winrate' : win_count[1],
        'loserate' : win_count[0] 
    }
    ch_win_df = ch_win_df.append(data,ignore_index=True)

# 챔피언별 상대 승률
tmp_df = c_df[c_df['championId'] == key]
champ_name = 'Riven' # 챔피언 이름

match_index = c_df.index[c_df['championName'] == champ_name].tolist() # key 챔피언 id로 고르기
for i in range(len(match_index)):
  a = c_df.loc[match_index[i]] # 해당 인덱스의 게임 정보
  tmp_df2 = c_df[c_df['gameId'] == a['gameId']] # gameID가 같은경우
  tmp_df2 = tmp_df2[tmp_df2['individualPosition'] == a['individualPosition']] # 같은 라인인 경우 
  if(i==0):
    champ_opp_df = tmp_df2
  else:
    champ_opp_df = champ_opp_df.append(tmp_df2)


champ_opp_df2 = champ_opp_df[champ_opp_df['championName'] != champ_name] # 내가 선택한 챔피언이 아닌 상대편의 정보만 
opp_champ_id = pd.DataFrame(champ_opp_df2['championName'].value_counts()) # 상대했던 챔피언들 count
opp_champ_id['win'] = 0
opp_champ_id.rename(columns={'championName':'matchCount'},inplace=True)
for i in range(len(champ_opp_df2)):
  tmp = champ_opp_df2.iloc[i]
  if(tmp['win'] == False):
    opp_champ_id.loc[tmp['championName'], 'win'] +=1


opp_champ_id['winrate'] = opp_champ_id['win'] / opp_champ_id['matchCount']
opp_champ_id.head()


  





,matchCount,win,winrate
Jayce,63,30,0.476190
Camille,56,33,0.589286
Fiora,44,20,0.454545
Kennen,43,19,0.441860
Graves,31,13,0.419355


In [ ]:
gm_df = gm_data

champion_df = gm_df.loc[:,['championId', 'championName']]
champion_df = champion_df.drop_duplicates() # 중복제거
champion_df = champion_df.sort_values('championId') # championid 순 정렬

id_list = champion_df['championId'].tolist()
id_dict = champion_df.set_index('championId').T.to_dict('list')
gm_win_df = pd.DataFrame(columns=['championId','championName', 'winrate', 'loserate'])

for key,value in id_dict.items(): # key = id value = name
  if(key < 888): #결측값 처리
    tmp_df = gm_df[gm_df['championId'] == key] 
    win_count = tmp_df['win'].value_counts(normalize=True)
    data = {
        'championId' : key,
        'championName' : value[0],
        'winrate' : win_count[1],
        'loserate' : win_count[0] 
    }
    gm_win_df = gm_win_df.append(data,ignore_index=True)
gm_win_df


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


,championId,championName,winrate,loserate
0,1,Annie,0.497143,0.502857
1,2,Olaf,0.490385,0.509615
2,3,Galio,0.524702,0.475298
3,4,TwistedFate,0.504742,0.495258
4,5,XinZhao,0.507958,0.492042
5,6,Urgot,0.448980,0.551020
6,7,Leblanc,0.482263,0.517737
7,8,Vladimir,0.494764,0.505236
8,9,FiddleSticks,0.480676,0.519324
9,10,Kayle,0.470930,0.529070


In [ ]:
m_df = m_data

champion_df = m_df.loc[:,['championId', 'championName']]
champion_df = champion_df.drop_duplicates() # 중복제거
champion_df = champion_df.sort_values('championId') # championid 순 정렬

id_list = champion_df['championId'].tolist()
id_dict = champion_df.set_index('championId').T.to_dict('list')
m_win_df = pd.DataFrame(columns=['championId','championName', 'winrate', 'loserate'])

for key,value in id_dict.items(): # key = id value = name
  if(key < 888): #결측값 처리
    tmp_df = m_df[m_df['championId'] == key] 
    win_count = tmp_df['win'].value_counts(normalize=True)
    data = {
        'championId' : key,
        'championName' : value[0],
        'winrate' : win_count[1],
        'loserate' : win_count[0] 
    }
    m_win_df = m_win_df.append(data,ignore_index=True)
m_win_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


,championId,championName,winrate,loserate
0,1,Annie,0.519317,0.480683
1,2,Olaf,0.485783,0.514217
2,3,Galio,0.521242,0.478758
3,4,TwistedFate,0.496815,0.503185
4,5,XinZhao,0.505222,0.494778
5,6,Urgot,0.512629,0.487371
6,7,Leblanc,0.484421,0.515579
7,8,Vladimir,0.495350,0.504650
8,9,FiddleSticks,0.496117,0.503883
9,10,Kayle,0.490928,0.509072
